In [1]:
import pandas as pd
import numpy as np
import sklearn
import pickle as pkl

import sys
sys.path.append("../src/")
import my_functions

In [5]:
input_file = "../data/raw/train.csv"
output_file = "../data/features/train_raul.pkl" 
my_functions.build_features_raul(input_file, output_file, split="training")

In [3]:
df.dtypes

Survived          bool
Pclass        category
Sex           category
Age            float64
SibSp            int64
Parch            int64
Ticket        category
Fare           float64
Embarked      category
Title         category
Is_Married        bool
Fsize             bool
TypeF         category
dtype: object

In [3]:
df.dtypes

Survived          bool
Pclass        category
Sex           category
Age            float64
SibSp            int64
Parch            int64
Ticket        category
Fare           float64
Embarked      category
Title         category
Is_Married        bool
Fsize             bool
TypeF         category
dtype: object

## Preprocess

In [2]:
def execute(input_file, output_file):
    data = pd.read_csv(input_file, sep = ";")
    data = data.dropna()
    del(data["Name"])
    del(data["Ticket"])
    del(data["Cabin"])
    

    data.to_csv(output_file)

In [9]:
input_file="../data/raw/val.csv"
output_file="../data/preprocess/last_val.csv"

In [10]:
execute(input_file=input_file,output_file=output_file)

## Build Features

In [11]:
def execute(input_file, output_file, force_write = True):
    """Builds features

    Args:
        input_file (str): input file.
    """
    df = pd.read_csv(input_file)

    df["Sex"] = df["Sex"].replace("male", 0)
    df["Sex"] = df["Sex"].replace("female", 1)

    embarked_dict = {}
    embarked_dict_values = 0
    for i in df.Embarked:
        if i in embarked_dict.keys():
            pass
        else:
            embarked_dict_values = embarked_dict_values + 1
            embarked_dict[i] = embarked_dict_values
    
    for i in embarked_dict.keys():
        df["Embarked"].replace(i, embarked_dict[i], inplace = True)

    df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
    df["IsAlone"] = 0
    df.loc[df["FamilySize"] == 1, "IsAlone"] = 1

    df.to_csv(output_file)

In [16]:
input_file="../data/preprocess/last_val.csv"
output_file="../data/features/last_val_buildfeatures.csv"

In [17]:
execute(input_file=input_file,output_file=output_file)

## Train

In [18]:
# Split the data for training.
df = pd.read_csv("../data/features/last_val_buildfeatures.csv")

y = df["Survived"]

tr_col = []
for c in df.columns: #Iterate over the dictionary of column names
    if c == "Survived":
        pass
    else:
        tr_col.append(c) #List of column names

In [21]:
df.dtypes

Unnamed: 0        int64
Unnamed: 0.1      int64
PassengerId       int64
Survived          int64
Pclass            int64
Sex               int64
Age             float64
SibSp             int64
Parch             int64
Fare            float64
Embarked          int64
FamilySize        int64
IsAlone           int64
dtype: object

In [22]:
df2=pd.read_csv("../data/features/val_RF_roche.csv")

In [23]:
df2.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked        object
FamilySize       int64
IsAlone          int64
dtype: object

In [19]:
# Create a classifier and select scoring methods.
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)


# # Fit full model and predict on both train and test.
# clf.fit(df[tr_col], y)
# preds = clf.predict(df[tr_col])
# metric_name = "train_accuracy" #Bad to put it here it is just a string
# metric_result = sklearn.metrics.accuracy_score(y, preds)

In [ ]:
#See what this does, no file data/model.pkl 

# model_pickle = open("data/model.pkl", 'wb')
# pkl.dump(clf, model_pickle)
# model_pickle.close()


In [24]:
# Return metrics and model.
info = ""
info = info + metric_name # No need to put info + metric_name
info = info + " for the model is "
info = info + str(metric_result)
print(info)

train_accuracy for the model is 0.9822485207100592


## Prediction

In [25]:
df = pd.read_csv("../output/val_buildfeatures.csv")

df.dropna(inplace = True)

target = df["Survived"]
del(df["Survived"])
    
# model_unpickle = open("data/model.pkl", 'rb')
# model = pkl.load(model_unpickle)
# model.close()

predictions = clf.predict(df)
# Reassign target (if it was present) and predictions.
df["prediction"] = predictions
df["target"] = target

ok = 0
for i in df.iterrows():
    if (i[1]["target"] == i[1]["prediction"]):
        ok = ok + 1

print("accuracy is", ok / df.shape[0])

accuracy is 0.7857142857142857


In [26]:
df.head(2)

,Unnamed: 0,Unnamed: 0.1,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,prediction,target
0,0,1,2,1,1,38.0,1,0,71.2833,1,2,0,1,1
1,1,3,4,1,1,35.0,1,0,53.1000,2,2,0,1,1
